In [1]:
from tsunami.constants import *
from tsunami.utils import read_yaml_file


In [3]:
import pandas as pd

In [4]:
schmea=read_yaml_file(file_path=r"C:\Users\Admin\OneDrive\Desktop\Tsunami_Git\Tsunami_Prediction_Pipeline\config\schema.yaml")

In [7]:
df=pd.read_csv(r"c:\\Users\\Admin\\OneDrive\\Desktop\\Tsunami_Git\\Tsunami_Prediction_Pipeline\\tsunami\\artifact\\data_ingestion\\2024-02-19-18-41-16\\train\\tsunami.csv")

In [18]:
df_type=list(map(lambda x:str(x).replace("dtype('","").replace("')",""),df.dtypes.values))